In [8]:
from refchecker.utils import get_model_batch_response
import os
import json
import re

# 设置API Key
os.environ['OPENAI_API_KEY'] = 'sk-proj-nkBZbCF2Y8sTEEVVme17T3BlbkFJkcxNNHrd95qcHIj76iMj'

result = []

# 模板
prompt_template = """ I would like you to help me measure the semantic similarity between two pieces of text. 
I will provide you with two texts: text1 and text2. 
You need to return two score between 0 and 10, with a higher score indicating a greater similarity between the two texts. 

Use the following format for your output:

### Scores between text1 and text2:
[score]

Now please give your assessment of the semantic similarity between the following texts:

### text1:
[text1]

### text2:
[text2]

"""

# 加载数据
data_path = 'data/300/raw.json'
with open(data_path, 'r') as f:
    data = json.load(f)

for i, d in enumerate(data):
    reference = d['reference']  
    text1 = d['text1']
    text2 = d['text2']

    # 生成第一个 prompt，替换文本
    prompt1 = prompt_template.replace('[text1]', reference).replace('[text2]', text1)

    # 获取第一个文本对的响应
    responses = get_model_batch_response(
        prompts=[prompt1],
        model='gpt-4o',
        temperature=0.5,
        max_new_tokens=500
    )
    response1 = responses[0]
    print(response1)
    assert 0

    # 定义提取分数的正则表达式
    pattern = re.compile(r'### Scores between text1 and text2:\s*(\d+)', re.DOTALL)

    # 提取第一个分数
    score1_match = pattern.search(response1)
    score1 = float(score1_match.group(1)) if score1_match else None

    # 生成第二个 prompt，替换文本
    prompt2 = prompt_template.replace('[text1]', reference).replace('[text2]', text2)

    # 获取第二个文本对的响应
    responses = get_model_batch_response(
        prompts=[prompt2],
        model='gpt-4o',
        temperature=0,
        max_new_tokens=500
    )
    response2 = responses[0]

    # 提取第二个分数
    score2_match = pattern.search(response2)
    score2 = float(score2_match.group(1)) if score2_match else None

    # 输出提取的分数
    print(f"Scores for pair {i+1}: {score1}, {score2}")
    # 将结果存储到列表中
    result.append([score1, score2])
 
# 保存结果到json文件
with open('data/300/gpt_score.json', 'w') as f:
    json.dump(result, f, indent=2)


### Scores between text1 and text2:
8.5

### Explanation:
Both texts provide a detailed, step-by-step guide on how to prepare for an important presentation, with significant overlap in the steps mentioned. Both texts emphasize understanding the audience, defining the objective, creating an outline, designing visual aids, and practicing the presentation. 

However, there are some differences:
- Text2 includes steps like "Research and gather materials" and "Anticipate questions," which are not explicitly mentioned in Text1.
- Text1 includes steps like "Get feedback" and "Rehearse in the actual presentation space," which are not explicitly mentioned in Text2.
- Text2 also emphasizes staying calm and confident during the presentation, which is implied but not explicitly stated in Text1.

Despite these differences, the core structure and intent of both texts are very similar, hence the high similarity score.


AssertionError: 

In [5]:
import json

with open('data/300/gpt_score.json', 'r') as f:
    gpt_data = json.load(f)

with open('data/300/raw.json', 'r') as f:
    raw_data = json.load(f)

correct = 0
sum = 80
for i, d in enumerate(raw_data):
    d['gpt_score1'] = gpt_data[i][0]
    d['gpt_score2'] = gpt_data[i][1]

    

    if d['gpt_score1'] > d['gpt_score2']:
        d['gpt_label'] = 0
    elif d['gpt_score1'] < d['gpt_score2']:
        d['gpt_label'] = 1
    else:
        d['gpt_label'] = 2  # equal
    
    if d['gt_labels'] == d['gpt_label']:
        correct += 1

print(f'Accuracy: {correct/sum}')   
    

Accuracy: 0.5
